In [1]:
import sys
from glob import glob
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import unique, join
from scipy import signal
from scipy.stats import pearsonr
from tqdm import tqdm
import pandas as pd
from scipy.optimize import curve_fit

In [2]:
from source_names_dict import source_names_dict, source_names_w_counterparts, source_names_readable
from get_closest_srcreg import get_src_region_dict
from uvot import read_uvotsource, calc_uvot_flags, filters, filter_colors, filter_markers
from xrt import load_xrt, rename_xrt_cols, curves
from lightcurve import LightCurve

In [3]:
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

In [4]:
# Parameters
plot_errors = False
simbad_name = 'NAME_NGC_1313_X-2'

uvot_filter   = 'U'
xrt_curve     = 'HARD'

xrt_rate      = 'RATE'
xrt_rate_err  = 'RATE_ERR'

uvot_rate     = 'COI_SRC_RATE'
uvot_rate_err = 'COI_SRC_RATE_ERR'


In [5]:
src_region_dict = get_src_region_dict()
local_name    = source_names_dict[simbad_name]
readable_name = source_names_readable[simbad_name]
closest_src   = src_region_dict[simbad_name]

Getting source region dict


In [6]:
# Load X-ray Data
tab_xrt_full = load_xrt(simbad_name=simbad_name, curve=xrt_curve, pandas=False)

# Load UV Data
fits_files = glob(f'../download_scripts/{local_name}/*uvotsource*fits*')
for f in fits_files:
    if closest_src.split('/')[-1][:-4] in f:
        tab_uvot = read_uvotsource(f)
tab_uvot = tab_uvot[tab_uvot['FILTER'] == uvot_filter]
tab_uvot = unique(tab_uvot, keys='OBSID')37.2166737.21667


IndexError: list index out of range

In [ ]:
tab_xrt_full

In [ ]:
tab_uvot

In [ ]:

# Join Data
tab_join = join(tab_uvot, tab_xrt_full , join_type='inner', keys='OBSID')
tab_join.sort('MJD_2')
# Get x,y data
t = tab_join['MJD_2']

x = tab_join[xrt_rate]
x_err = tab_join[xrt_rate_err]

y = tab_join[uvot_rate]
y_err = tab_join[uvot_rate_err]


# Caclulate pearson
r, p_val = pearsonr(x,y)

# calculate ccf
lags = np.arange(-len(x) + 1, len(x))
c = signal.correlate(x / np.std(x), y / np.std(y), 'full') 
c = c / len(x)
lag_max = lags[np.argmax(c)]

#calculate autocorrelations
import statsmodels.api as sm
acf_y = sm.tsa.acf(y, nlags=2*len(y))
acf_x = sm.tsa.acf(x, nlags=2*len(x))


In [ ]:
# Plot lightcurve
fig, ax = plt.subplots(1, 1, figsize=(10,10/3))
ax2 = ax.twinx()
ax.errorbar(t, x, yerr=x_err, lw=1.0, label=f'XRT {xrt_curve} | {xrt_rate}', ls='none', marker='.', color='black', capsize=1.0)
ax2.errorbar(t, y, yerr=y_err, lw=1.0, label=f'UVOT {uvot_filter} | {uvot_rate}', ls='none', marker='.', color='violet', capsize=1.0)
ax.set_ylabel(f'{xrt_curve} {xrt_rate} (ct/s)')
ax2.set_ylabel(f'{uvot_filter} {uvot_rate} (ct/s)')
ax.set_title(readable_name)
ax.legend(loc='upper left')
ax2.legend(loc='upper right')
ax.set_xlabel('Time (MJD)')
plt.tight_layout()
plt.savefig(f'correlation_outputs/plots/{simbad_name}_{xrt_curve}_{uvot_filter}_lc_mjd.png')
plt.savefig(f'correlation_outputs/plots/{simbad_name}_{xrt_curve}_{uvot_filter}_lc_mjd.pdf')

# Plot lightcurve (no time)
fig, ax = plt.subplots(1, 1, figsize=(10,10/3))
ax2 = ax.twinx()
ax.plot(range(len(x)), x, lw=1.0, label=f'XRT {xrt_curve} | {xrt_rate}', ls='-', marker='.', color='black')
ax2.plot(range(len(y)), y, lw=1.0, label=f'UVOT {uvot_filter} | {uvot_rate}', ls='-', marker='.', color='violet')
ax.set_ylabel(f'{xrt_curve} {xrt_rate} (ct/s)')
ax2.set_ylabel(f'{uvot_filter} {uvot_rate} (ct/s)')
ax.set_title(readable_name)
ax.legend(loc='upper left')
ax2.legend(loc='upper right')
ax.set_xlabel('# Datapoint')
plt.tight_layout()
plt.savefig(f'correlation_outputs/plots/{simbad_name},{xrt_curve},{uvot_filter},lc_seq.png')
plt.savefig(f'correlation_outputs/plots/{simbad_name},{xrt_curve},{uvot_filter},lc_seq.pdf')

# Plot correlation (No errors)
plt.figure(figsize=(5,5))
plt.scatter(x, y, marker='x', color='black')
plt.title(f'{readable_name} | $r$ = {r:.2f}')
plt.xlabel(f'{xrt_curve} {xrt_rate} (ct/s)')
plt.ylabel(f'{uvot_filter} {uvot_rate} (ct/s)')
plt.tight_layout()
plt.savefig(f'correlation_outputs/plots/{simbad_name},{xrt_curve},{uvot_filter},corr.png')
plt.savefig(f'correlation_outputs/plots/{simbad_name},{xrt_curve},{uvot_filter},corr.pdf')

# Plot Correlation with errors
plt.figure(figsize=(5,5))
plt.errorbar(x, y, xerr=x_err, yerr=y_err,  lw=1.0, capsize=1.0, ls='none', color='black')
plt.title(f'{readable_name} | $r$ = {r:.2f}')
plt.xlabel(f'{xrt_curve} {xrt_rate} (ct/s)')
plt.ylabel(f'{uvot_filter} {uvot_rate} (ct/s)')
plt.tight_layout()
plt.savefig(f'correlation_outputs/plots/{simbad_name},{xrt_curve},{uvot_filter},corr_err.png')
plt.savefig(f'correlation_outputs/plots/{simbad_name},{xrt_curve},{uvot_filter},corr_err.pdf')

# Plot CCF
plt.figure(figsize=(5,5))
plt.plot(lags, c, label='CCF', color='black')
plt.axvline(lag_max, color='red', label=f'{lag_max}')
plt.legend()
plt.title(f'{readable_name}')
plt.xlabel('Lag (datapoints)')
plt.ylabel('CCF')
plt.tight_layout()
plt.savefig(f'correlation_outputs/plots/{simbad_name},{xrt_curve},{uvot_filter},corr_ccf.png')
plt.savefig(f'correlation_outputs/plots/{simbad_name},{xrt_curve},{uvot_filter},corr_ccf.pdf')

# Plot autocorrelations
plt.figure(figsize=(5,5))
plt.title(f'{readable_name}')
plt.plot(acf_x, label=f'ACF UVOT')
plt.plot(acf_y, label=f'ACF XRT')
plt.legend()
plt.tight_layout()
plt.savefig(f'correlation_outputs/plots/{simbad_name},{xrt_curve},{uvot_filter},corr_acf.png')
plt.savefig(f'correlation_outputs/plots/{simbad_name},{xrt_curve},{uvot_filter},corr_acf.pdf')

In [ ]:
def line(x, m, c):
    return m*x+c

In [ ]:
plt.figure(figsize=(4,4))
all_mc_res = []
for i in tqdm(range(10000)):
    mc_res = {}
    x_samp = np.random.normal(loc=x, scale=x_err)
    y_samp = np.random.normal(loc=y, scale=y_err)
    r, p_val = pearsonr(x_samp,y_samp)
    p_opt, p_cov = curve_fit(line, x_samp, y_samp)
    m, c = p_opt
    
    mc_res['r'] = r
    mc_res['m'] = m
    mc_res['c'] = c
    all_mc_res.append(mc_res)
    
df_mc_res = pd.DataFrame(all_mc_res)
df_mc_res.to_csv(f'correlation_outputs/tables/{simbad_name},{xrt_curve},{uvot_filter},line_mc.csv', index=False)


# Calcultae best fit values and errors
r_best = df_mc_res['r'].mean()
r_std  = df_mc_res['r'].std()
m_best = df_mc_res['m'].mean()
m_std  = df_mc_res['m'].std()
c_best = df_mc_res['c'].mean()
c_std  = df_mc_res['c'].std()


In [ ]:
ax = df_mc_res.hist(bins=100, grid=False)
plt.savefig(f'correlation_outputs/plots/{simbad_name},{xrt_curve},{uvot_filter},corr_dist.eps')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,5), facecolor='white')
ax.scatter(x, y, marker='x', color='grey', alpha=0.8)     

xmin, xmax = ax.get_xlim()
xrange = np.linspace(xmin, xmax, 10)


# Plot best fit line
ax.plot(xrange, line(xrange, m_best, c_best), color='red', label=f'y={m_best:.2f}x+{c_best:.2f}')
ax.set_xlim(xmin, xmax)
ax.legend(loc='upper right')

# Plot binned data
xmax = np.max(x)
xmin = np.min(x)
nbins = 9
xbins = np.linspace(xmin, xmax, nbins)

for i in range(nbins-1):
    xlow = xbins[i]
    xhigh = xbins[i+1]
    xs = x[(x>xlow) & (x<xhigh)]
    ys = y[(x>xlow) & (x<xhigh)]
    
    x_mean = np.mean(xs)
    x_err = np.std(xs) / np.sqrt(len(xs))
    y_mean = np.mean(ys)
    y_err = np.std(ys) / np.sqrt(len(ys))
    ax.errorbar(x_mean, y_mean, yerr=y_err, color='black', capsize=1.0, lw=1.0, marker='.', label='Binned')
    
    xmin, xmax = ax.get_xlim()
    xrange = np.linspace(xmin, xmax, 10)
    

ax.set_xlabel(f'XRT {xrt_curve} {xrt_rate} (ct/s)')
ax.set_ylabel(f'UVOT {uvot_filter} RATE (ct/s)')
ax.set_title(f'{readable_name} | $r$ = {r_best:.2f} $\pm$ {r_std:.2f}')
plt.savefig(f'correlation_outputs/plots/{simbad_name},{xrt_curve},{uvot_filter},corr_fit.png')
plt.savefig(f'correlation_outputs/plots/{simbad_name},{xrt_curve},{uvot_filter},corr_fit.eps')